In [11]:
"""
Isolation Forest 예측 스크립트
- 학습된 모델로 새로운 데이터 예측
- 이상 점수 및 레이블 반환
"""

import os
import pickle
import numpy as np
import pandas as pd
from datetime import datetime

# =====================================
# 설정
# =====================================

# 입력
INPUT_FEATURES = 'data/processed/1118_features_misalignment_yellow.csv'  # 예측할 데이터

# 모델 경로
MODEL_PATH = 'models/isolation_forest.pkl'
SCALER_PATH = 'models/scaler_if.pkl'

# 출력
OUTPUT_DIR = 'results/isolation_forest'
OUTPUT_FILE = 'predictions.csv'

os.makedirs(OUTPUT_DIR, exist_ok=True)

# =====================================
# 1. 모델 및 스케일러 로드
# =====================================
print("="*70)
print("📦 Loading Model and Scaler")
print("="*70)

with open(MODEL_PATH, 'rb') as f:
    model = pickle.load(f)
print(f"✅ Model loaded: {MODEL_PATH}")

with open(SCALER_PATH, 'rb') as f:
    scaler = pickle.load(f)
print(f"✅ Scaler loaded: {SCALER_PATH}")

# =====================================
# 2. 데이터 로드
# =====================================
print("\n" + "="*70)
print("📂 Loading Test Data")
print("="*70)

df = pd.read_csv(INPUT_FEATURES)
print(f"✅ Loaded {len(df)} samples")

# 메타데이터 분리
metadata_cols = ['window_id', 'start_time', 'end_time', 'n_samples']
feature_cols = [col for col in df.columns if col not in metadata_cols]

X = df[feature_cols].values
print(f"✅ Features: {len(feature_cols)}")

# =====================================
# 3. 정규화
# =====================================
X_scaled = scaler.transform(X)
print(f"✅ Data normalized")

# =====================================
# 4. 예측
# =====================================
print("\n" + "="*70)
print("🔍 Making Predictions")
print("="*70)

# 이상 점수
anomaly_scores = model.score_samples(X_scaled)

# 예측 (-1: 이상, 1: 정상)
predictions = model.predict(X_scaled)

# 통계
n_normal = np.sum(predictions == 1)
n_anomaly = np.sum(predictions == -1)

print(f"Predictions:")
print(f"  Normal (1):   {n_normal:5d} ({n_normal/len(predictions)*100:.1f}%)")
print(f"  Anomaly (-1): {n_anomaly:5d} ({n_anomaly/len(predictions)*100:.1f}%)")
print(f"\nAnomaly scores:")
print(f"  Mean:   {anomaly_scores.mean():.6f}")
print(f"  Std:    {anomaly_scores.std():.6f}")
print(f"  Min:    {anomaly_scores.min():.6f}")
print(f"  Max:    {anomaly_scores.max():.6f}")

# =====================================
# 5. 결과 저장
# =====================================
print("\n" + "="*70)
print("💾 Saving Results")
print("="*70)

# 결과 DataFrame 생성
df_results = df.copy()
df_results['anomaly_score'] = anomaly_scores
df_results['prediction'] = predictions
df_results['prediction_label'] = ['Normal' if p == 1 else 'Anomaly' for p in predictions]
df_results['prediction_time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# 저장
output_path = os.path.join(OUTPUT_DIR, OUTPUT_FILE)
df_results.to_csv(output_path, index=False)
print(f"✅ Results saved: {output_path}")

# =====================================
# 6. 이상 샘플 확인
# =====================================
if n_anomaly > 0:
    print("\n" + "="*70)
    print(f"⚠️  Detected Anomalies: {n_anomaly}")
    print("="*70)
    
    # 이상으로 판별된 샘플
    anomaly_indices = np.where(predictions == -1)[0]
    
    # 점수 기준 상위 10개 (가장 이상한 것)
    top_k = min(10, n_anomaly)
    top_anomaly_indices = anomaly_indices[np.argsort(anomaly_scores[anomaly_indices])[:top_k]]
    
    print(f"\nTop {top_k} anomalies (lowest scores):")
    for i, idx in enumerate(top_anomaly_indices, 1):
        score = anomaly_scores[idx]
        window_id = df_results.iloc[idx]['window_id']
        start_time = df_results.iloc[idx]['start_time']
        print(f"  {i:2d}. Window {window_id:3.0f} (time: {start_time:6.1f}s) - Score: {score:.6f}")
else:
    print("\n✅ No anomalies detected - all samples are normal")

# =====================================
# 7. 요약
# =====================================
print("\n" + "="*70)
print("✅ Prediction Complete")
print("="*70)
print(f"\n[Results]")
print(f"  • Output file:   {output_path}")
print(f"  • Total samples: {len(predictions)}")
print(f"  • Normal:        {n_normal} ({n_normal/len(predictions)*100:.1f}%)")
print(f"  • Anomaly:       {n_anomaly} ({n_anomaly/len(predictions)*100:.1f}%)")
print("\n" + "="*70)

📦 Loading Model and Scaler
✅ Model loaded: models/isolation_forest.pkl
✅ Scaler loaded: models/scaler_if.pkl

📂 Loading Test Data
✅ Loaded 719 samples
✅ Features: 77
✅ Data normalized

🔍 Making Predictions
Predictions:
  Normal (1):     383 (53.3%)
  Anomaly (-1):   336 (46.7%)

Anomaly scores:
  Mean:   -0.509271
  Std:    0.055461
  Min:    -0.686043
  Max:    -0.407005

💾 Saving Results
✅ Results saved: results/isolation_forest\predictions.csv

⚠️  Detected Anomalies: 336

Top 10 anomalies (lowest scores):
   1. Window 372 (time: 1860.0s) - Score: -0.686043
   2. Window 384 (time: 1920.0s) - Score: -0.685992
   3. Window 336 (time: 1680.0s) - Score: -0.685264
   4. Window  36 (time:  180.0s) - Score: -0.680816
   5. Window 444 (time: 2220.0s) - Score: -0.679061
   6. Window 180 (time:  900.0s) - Score: -0.677808
   7. Window 144 (time:  720.0s) - Score: -0.676169
   8. Window 108 (time:  540.0s) - Score: -0.676035
   9. Window 504 (time: 2520.0s) - Score: -0.674832
  10. Window 264 